In [8]:
# Installa la libreria insanely-fast-whisper
!pipx install insanely-fast-whisper --force --pip-args="--ignore-requires-python"
!pip install transformers datasets librosa
!pip install soundfile

import subprocess
import librosa
import soundfile as sf
import numpy as np
import gc
import torch
import os
# os.environ["CUDA_VISIBLE_DEVICES"] = ""
# torch.device('cpu')

# Definisci i possibili tag per la classificazione
from transformers import pipeline
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")
candidate_labels = ["finanza", "salute", "tecnologia", "intrattenimento"]

# Carica il file audio con librosa
from google.colab import files
uploaded = files.upload()
file_name = list(uploaded.keys())[0]

# Usa librosa per caricare l'audio (puoi anche fare direttamente riferimento al file)
speech, sample_rate = librosa.load(file_name, sr=16000, mono=True)  # Forza il sample rate a 16000 Hz e converte in mono

duration = librosa.get_duration(y=speech, sr=sample_rate)
print(f"Durata del file audio: {duration} secondi")

# Salva temporaneamente l'audio usando soundfile
temp_audio_file = 'temp_audio.wav'
sf.write(temp_audio_file, speech, sample_rate)

# Funzione per eseguire la trascrizione con insanely-fast-whisper
def transcribe_audio_with_whisper(audio_file):
    # Rimuovi il flag --device-id
    command = f"insanely-fast-whisper --file-name {audio_file} --language it --task transcribe"
    result = subprocess.run(command, shell=True, capture_output=True, text=True)

    if result.returncode == 0:
        return result.stdout
    else:
        print(f"Errore durante la trascrizione: {result.stderr}")
        return None



# Trascrivi l'audio usando insanely-fast-whisper
transcription = transcribe_audio_with_whisper(temp_audio_file)
if transcription:
    print("Trascrizione completa:", transcription)

    # Classificazione della trascrizione
    classification = classifier(transcription, candidate_labels)
    most_relevant_tag = classification['labels'][0]  # Estrai il tag più rilevante
    print("Tag rilevante:", most_relevant_tag)
else:
    print("Errore nella trascrizione.")

# Pulisci file temporanei
import os
os.remove(temp_audio_file)

gc.collect()  # Forza la liberazione della memoria


/bin/bash: line 1: pipx: command not found


Saving [Giovanni Calia]_302-155_20240612104659(68).wav to [Giovanni Calia]_302-155_20240612104659(68).wav
Durata del file audio: 5.632 secondi
Errore durante la trascrizione: 2024-09-27 10:05:06.113138: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-27 10:05:06.150341: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-27 10:05:06.161017: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-27 10:05:07.744705: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Traceback (most recent call last):
  File "/usr/local/bin/insanely

73